# First Assignment

# First question : the postal code of each neighborhood along with the borough name and neighborhood name

The objective of this code is to scrape a wikipedia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) in order to get the boroughs and neighboroughs in Toronto 

In [23]:
import requests
import pandas as pd
import re

we request the page with the library **requests** 

In [24]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&action=edit"
response = requests.get(url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&action=edit")

In [25]:
response.status_code

200

the request worked!

We need to split the text in order to identify the valued information. 

In [26]:
data_info = response.text.split("{")[16].split("\n")[5:]

In [27]:
data_info[0:8]

['| M1A', '| Not assigned', '| ', '|-', '| M2A', '| Not assigned', '| ', '|-']

In [28]:
Postal_code=list()
Borough=list()
Neighborhood=list()

for i in range(0,len(data_info)-2,4):
    Postal_code = Postal_code + [data_info[i].replace("| ","")]
    Borough = Borough + [data_info[i+1].replace("| ","")]
    Neighborhood = Neighborhood + [data_info[i+2].replace("| ","")]

In [29]:
df=pd.DataFrame({"PostalCode":Postal_code,"Borough":Borough,"Neighborhood":Neighborhood})
df = df[~(df.Borough=='Not assigned')].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


let's put a comma between Neighborhood from the same Postal Code

In [30]:
aggregation = {"Neighborhood": lambda x :  ", ".join(x)}
df = df.groupby(["PostalCode","Borough"]).agg(aggregation).reset_index()

df.Neighborhood = [x.replace(" /",",") for x in df.Neighborhood]

In [31]:
df[df.PostalCode=='M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [32]:
df[df.Neighborhood=='Not assigned'].index.values

array([], dtype=int64)

it seems that there is no Neighborhood with **Not assigned**. However, if there were this would be the code to change their values.

In [33]:
for values in df.index.values:
    PC,Bor,Neigh = df.loc[values]
    if Neigh == 'Not assigned':
        df.loc[values,"Neighborhood"]=Bor

In [34]:
df.shape

(103, 3)

## Second question: Longitude and Latitude

In [35]:
#!pip install geocoder
import geocoder # import geocoder

In [36]:
def get_coordonnate(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    iterative = 0
    # loop until you get the coordinates
    while((lat_lng_coords is None) and (iterative<50)):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        iterative = iterative + 1
        print(iterative)
    return lat_lng_coords


In [37]:
#coords = [get_coordonnate(x) for x in df.PostalCode]

In [38]:
#df["Latitude"] = [x[0] for x in coords]
#df["Longitude"] = [x[1] for x in coords]
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

We import the csv file with the coordinates

In [39]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
df_1 = df.merge(df_data_0,left_on="PostalCode",right_on="Postal Code").drop(columns="Postal Code")
df_1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
